In [1]:
!pip install crewai earthengine-api

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Set OpenAI API Key as environment variable
import os
os.environ['OPENAI_API_KEY'] = 'Add your key here'  # Replace with your actual OpenAI API key

In [4]:
# Import CrewAI and Earth Engine
import ee
import crewai
from crewai.agent import Agent
from pydantic import Field, ValidationError  # Import Field from pydantic

# CrewAI Agent for exporting Sentinel-2 image to Google Drive
class Sentinel2ExportAgent(Agent):
    project_id: str = Field(..., description="Project ID for Earth Engine")  # Add project_id as a field

    def __init__(self, name, role, goal, backstory, project_id):
        try:
          # Initialize the agent with the required fields (without passing project_id to the parent)
          super().__init__(name=name, role=role, goal=goal, backstory=backstory, project_id=project_id)
        except ValidationError as e:
          print(e)
          return

        # Store the project_id internally (do not pass it to the parent constructor)
        self.project_id = project_id

        # Authenticate and initialize Earth Engine
        self.authenticate_and_initialize_earth_engine()

    def authenticate_and_initialize_earth_engine(self):
        try:
            # Authenticate the user
            ee.Authenticate()  # This will prompt OAuth authentication

            # Initialize Earth Engine with the specified project ID
            ee.Initialize(project=self.project_id)
            #self.log(f"Earth Engine initialized with project: {self.project_id}")
        except Exception as e:
            self.log(f"Error during authentication or initialization: {e}")

    def export_sentinel2_image_to_drive(self, latitude, longitude, start_date, end_date, image_name):
        try:
            # Define a point around which to get the image
            point = ee.Geometry.Point([longitude, latitude])

            # Load the Sentinel-2 image collection and filter by date and location
            collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                          .filterBounds(point)
                          .filterDate(start_date, end_date)
                          .sort('CLOUD_COVER', True))  # Sort by cloud cover to get the least cloudy image

            # Get the first image in the filtered collection
            image = collection.first()

            # Select relevant bands (e.g., RGB bands: B4, B3, B2)
            selected_bands = image.select(['B4', 'B3', 'B2'])

            # Define the region (smaller bounding box around the point, e.g., 1 km buffer)
            region = point.buffer(1000).bounds().getInfo()['coordinates']

            # Create an export task to Google Drive
            task = ee.batch.Export.image.toDrive(
                image=selected_bands,
                description=image_name,
                folder='EarthEngineImages',  # Folder name in Google Drive
                fileNamePrefix=image_name,
                region=region,
                scale=30,  # Resolution in meters
                crs='EPSG:4326',  # Coordinate Reference System
                fileFormat='GeoTIFF'
            )

            # Start the export task
            task.start()
            #self.log(f"Export task started to Google Drive for {image_name}.")
        except Exception as e:
            self.log(f"Error during export: {e}")

In [5]:
# Main code execution
if __name__ == "__main__":
    # Set up project details
    project_id = 'genai-agent-hack-2024'  # Replace with your actual project ID

    # Define the role, goal, and backstory for the agent
    role = "Data Analyst"
    goal = "Export Sentinel-2 imagery from Earth Engine to Google Drive"
    backstory = "The agent assists in data analysis by exporting high-resolution satellite imagery for further analysis."

    # Create and run the CrewAI agent
    sentinel2_agent = Sentinel2ExportAgent(
        name="Sentinel2ExportAgent",
        role=role,
        goal=goal,
        backstory=backstory,
        project_id=project_id
    )

    # Example usage: export a Sentinel-2 image for a specific location and date
    latitude = 37.7749   # Latitude for San Francisco
    longitude = -122.4194  # Longitude for San Francisco
    start_date = '2021-06-01'
    end_date = '2021-06-30'
    image_name = 'sentinel2_sf'

    sentinel2_agent.export_sentinel2_image_to_drive(latitude, longitude, start_date, end_date, image_name)